In [5]:
from sklearn.model_selection import train_test_split
import PIL.Image as Image
import matplotlib.pylab as plt
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
import tensorflow_hub as hub
import datetime
import keras
import base_functions as funcs
import numpy as np
import lightgbm as lgbm
data, labels = funcs.ReadCSV('./datasets/creditcard.csv', mark='Class')
data = data.drop(columns=['Time'])

In [19]:
##Получаем значимость переменных из бустинга

split_index=int(len(data)*0.8)

data_train, data_test, labels_train, labels_test = (data.iloc[0:split_index], data.iloc[split_index:],
                                                    labels.iloc[0:split_index], labels.iloc[split_index:])
lgbmClassifier = lgbm.LGBMClassifier(objective='binary', n_estimators=600, metric='prc',
                                     class_weight='balanced',   learning_rate=0.055   )

lgbmClassifier.fit(data_train , labels_train)
feat_imp=[]
for i in enumerate(data.columns):
    feat_imp.append([i[1], lgbmClassifier.feature_importances_[i[0]]])
feat_imp.sort()
sorted_importances=sorted(feat_imp, key=lambda feat: feat[1])#.reverse()   # sort by importance
for i in sorted_importances:
    print(i[0],'\t' ,i[1])


[LightGBM] [Info] Number of positive: 417, number of negative: 227428
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010148 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7395
[LightGBM] [Info] Number of data points in the train set: 227845, number of used features: 29
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

In [ ]:
##меняем порядок фич по значимости

data=data.reindex(columns=['V14','V4','V26','Amount','V12','V28','V24','V22','V11','V3','V10','V7','V25','V1','V5','V18','V8','V13','V27','V16','V23','V6','V19','V21','V15','V20','V9','V2','V17' ])

labels = np.array(labels)[0:]

In [20]:
##дробим на выборки по времени

length=len(data)
split_index=int(length*0.75)
split_index_val=int(length*0.8)
data_train, data_val, data_test, labels_train,labels_val, labels_test = \
    (data.iloc[0:split_index], data.iloc[split_index:split_index_val],data.iloc[split_index_val:],
                          labels[0:split_index], labels[split_index:split_index_val], labels[split_index_val:])
print("train:", len(data_train), "val:", len(data_val), "test:", len(data_test))

train: 213605 val: 14240 test: 56962


In [21]:
##стандартизируем данные
data_train, data_test, labels_train, labels_test =train_test_split(data, labels,
                                                                  test_size=0.2, random_state=4)
data_val,labels_val=data_test,labels_test
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(data_train)
data_train, data_val, data_test = scaler.transform(data_train), scaler.transform(data_val), scaler.transform(data_test)


In [22]:
##Функции конвертации в тензор 

def shuffle_Vector(vector): ##функция для задания шаффла, не используется , поэтому строка шаффла закомментирована
    copy_Vec=np.copy(vector)
    #np.random.shuffle(np.copy(copy_Vec))
    return copy_Vec

def my_func(arg):
    matrix = [shuffle_Vector(arg) for i in range(len(arg))]
    matrix=arg
    arg = tf.convert_to_tensor(np.concatenate([matrix], -1).reshape(1, 29), dtype=tf.float16)
    return arg

def convert_to_tensors(data):
    converted_data=[]
    counter=1
    for i in data:
        if counter%2000==0: print(counter)
        counter+=1
        converted_data.append(my_func(np.concatenate([i],axis=0)))
    return converted_data


In [23]:

data_train= convert_to_tensors(data_train)
data_val = convert_to_tensors(data_val)
data_test= convert_to_tensors(data_test)
labels_train=labels_train.reshape(len(labels_train),1)
labels_val=labels_val.reshape(len(labels_val),1)
labels_test=labels_test.reshape(len(labels_test),1)


2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
58000
60000
62000
64000
66000
68000
70000
72000
74000
76000
78000
80000
82000
84000
86000
88000
90000
92000
94000
96000
98000
100000
102000
104000
106000
108000
110000
112000
114000
116000
118000
120000
122000
124000
126000
128000
130000
132000
134000
136000
138000
140000
142000
144000
146000
148000
150000
152000
154000
156000
158000
160000
162000
164000
166000
168000
170000
172000
174000
176000
178000
180000
182000
184000
186000
188000
190000
192000
194000
196000
198000
200000
202000
204000
206000
208000
210000
212000
214000
216000
218000
220000
222000
224000
226000
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000


AttributeError: 'Series' object has no attribute 'reshape'

In [25]:
##варианты смещений

initial_bias = np.log([sum(labels_train)/(len(labels_train)-sum(labels_train))])
initial_bias = tf.keras.initializers.Constant(initial_bias)

In [28]:
##веса классов
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced',
                                                 classes=np.unique(labels_train), y=labels_train)


In [ ]:
## обучение модели
model = models.Sequential()
model.add(layers.Conv2D(32, (1, 8), activation='relu', input_shape=(1,29 ,1)))
#model.add(layers.MaxPooling2D((1, 2)))
model.add(layers.Conv2D(16, (1, 8), activation='relu'))
#model.add(layers.Dropout(0.05))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))
                            #, bias_initializer=initial_bias

print(model.summary())
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc',
    verbose=1,
    patience=6,
    mode='max',
    restore_best_weights=True)
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss_weights=class_weights,
             # loss=tf.keras.losses.BinaryFocalCrossentropy(from_logits=True ),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[ keras.metrics.TruePositives(name='tp'),
                                      keras.metrics.FalsePositives(name='fp'),
                                      keras.metrics.TrueNegatives(name='tn'),
                                      keras.metrics.FalseNegatives(name='fn'),
                                      keras.metrics.BinaryAccuracy(name='accuracy'),
                                      keras.metrics.Precision(name='precision'),
                                      keras.metrics.Recall(name='recall'),
                                      keras.metrics.AUC(name='auc'),
                                      keras.metrics.AUC(name='prc', curve='PR')
                      ],
               )

history = model.fit(tf.stack(data_train), labels_train, epochs=150, batch_size=72,
                    steps_per_epoch=2500,
                    validation_data=(tf.stack(data_val), labels_val), callbacks=[early_stopping],
                    workers=12, use_multiprocessing=True)



model.save_weights('./checkpoints/my_checkpoint')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 1, 22, 32)         288       
                                                                 
 conv2d_3 (Conv2D)           (None, 1, 15, 16)         4112      
                                                                 
 flatten_1 (Flatten)         (None, 240)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 241       
                                                                 
Total params: 4641 (18.13 KB)
Trainable params: 4641 (18.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/150



C:\Users\bobor\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\backend.py:5818: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2500/2500 [==============================] - 11s 4ms/step - loss: 1.5324 - tp: 175.0000 - fp: 91.0000 - tn: 179608.0000 - fn: 126.0000 - accuracy: 0.9988 - precision: 0.6579 - recall: 0.5814 - auc: 0.8581 - prc: 0.5135 - val_loss: 0.5149 - val_tp: 84.0000 - val_fp: 14.0000 - val_tn: 56847.0000 - val_fn: 17.0000 - val_accuracy: 0.9995 - val_precision: 0.8571 - val_recall: 0.8317 - val_auc: 0.9686 - val_prc: 0.8316
Epoch 2/150
2500/2500 [==============================] - 8s 3ms/step - loss: 0.5365 - tp: 235.0000 - fp: 41.0000 - tn: 179605.0000 - fn: 84.0000 - accuracy: 0.9993 - precision: 0.8514 - recall: 0.7367 - auc: 0.9253 - prc: 0.7620 - val_loss: 0.3736 - val_tp: 84.0000 - val_fp: 13.0000 - val_tn: 56848.0000 - val_fn: 17.0000 - val_accuracy: 0.9995 - val_precision: 0.8660 - val_recall: 0.8317 - val_auc: 0.9644 - val_prc: 0.8643
Epoch 3/150
2500/2500 [==============================] - 9s 3ms/step - loss: 0.4909 - tp: 234.0000 - fp: 39.0000 - tn: 179614.0000 - fn: 78.0000 - accuracy:

In [ ]:
##графики
from sklearn.metrics import PrecisionRecallDisplay, precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
predicts=model.predict(tf.stack(data_test))
print(predicts)
print('average_precision_score: ' , average_precision_score(labels_test, predicts))
print('roc_auc_score: ' , roc_auc_score(labels_test, predicts))

prec, recall, _ = precision_recall_curve(labels_test, predicts, pos_label=1)
pr_display = PrecisionRecallDisplay(precision=prec, recall=recall).plot()


import matplotlib.pyplot as plt

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
def plot_metrics(history):
  metrics = ['loss', 'prc', 'precision', 'recall']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend();

plot_metrics(history)